<a href="https://colab.research.google.com/github/coderAllDay/FaceRecognition/blob/master/FaceVerifica_Recogni_tion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [107]:
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import sys
from fr_utils import *
from inception_blocks_v2 import *
import matplotlib.pyplot as plt
%matplotlib inline
%autoreload 2
%reload_ext autoreload

np.set_printoptions(threshold=sys.maxsize)

In [108]:
FRmodel = faceRecoModel(input_shape = (3,96,96))

In [109]:
print("Total Parameters :", FRmodel.count_params())

Total Parameters : 3743280


In [110]:
def triplet_loss(y_true, y_pred, alpha = 0.2):
  anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]

  pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,positive)))
  neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor,negative)))

  basic_loss = tf.add(tf.subtract(pos_dist,neg_dist),alpha)
  loss = tf.maximum(tf.reduce_mean(basic_loss), 0.0)

  return loss

In [111]:
with tf.compat.v1.Session() as test:
  tf.random.set_seed(1)
  y_true = (None, None, None)
  y_pred = (
            tf.random.normal([3,128], mean = 6, stddev= 0.1, seed = 1),
            tf.random.normal([3,128], mean = 1, stddev= 1, seed = 1),
            tf.random.normal([3,128], mean = 3, stddev= 4, seed = 1)
            )
  loss = triplet_loss(y_true, y_pred)
  print("Loss = " + str(loss.eval()))

Loss = 350.02734


In [112]:
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])

In [113]:
load_weights_from_FaceNet(FRmodel)

FileNotFoundError: ignored

In [119]:
database = {}
database["danielle"] = img_to_encoding("/content/danielle.png", FRmodel)
database["younes"] = img_to_encoding("/content/younes.jpg", FRmodel)
database["tian"] = img_to_encoding("/content/tian.jpg", FRmodel)
database["andrew"] = img_to_encoding("/content/andrew.jpg", FRmodel)
database["kian"] = img_to_encoding("/content/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("/content/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("/content/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("/content/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("/content/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("/content/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("/content/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("/content/arnaud.jpg", FRmodel)

TypeError: ignored

In [118]:
def verify(image_path, identity,database, model):
  encoding = img_to_encoding(image_path, model)
  dist = np.linalg.norm(encoding - database[identity])

  if dist < 0.7:
    print("It's" + str(identity) + ",Welcome In!")
    door_open = True
  else:
    print("It's not" + str(identity) + ",please get lost")
    door_open = False
  return dist, door_open

In [116]:
verify("images/camera_0.jpg", "younes", database, FRmodel)

TypeError: ignored

In [ ]:
verify("images/camera_0.jpg", "kian",database, FRmodel)

TypeError: ignored

In [ ]:
def who_it_is(image_path, database, model):
  encoding = img_to_encoding(image_path, model)
  min_dist = 100
  for (name, db_enc) in database.items():
    dist = np.linalg.norm(encoding - db_enc)
    if dist < min_dist:
      min_dist = dist
      identity = name
  if min_dist > 0.7:
    print("Not in the Database")
  else:
    print("It's" + str(identity) + ",the distance is" + str(min_dist))
  return min_dist, identity

In [ ]:
verify("images/camera_0.jpg", database, FRmodel)

TypeError: ignored